In [1]:
from tensorflow import keras
from tensorflow.keras.models import load_model
import pickle

from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, auc
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import RocCurveDisplay
from scipy import interp
from sklearn.model_selection import StratifiedKFold

import numpy as np
from keras.utils import np_utils
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import cycle
import pandas as pd
import glob 
import os 

In [37]:
models1 = glob.glob('/Users/home/Documents/Respiratory Illness AI/randomness_test/run2/*/') 

In [31]:
fh = open("/Users/home/Documents/Respiratory Illness AI/randomness_test/recording/test2/asthma_others2_224_224_3", 'rb')
dataset = pickle.load(fh)

In [4]:
def train_val(dataset):
    input_shape = (224, 224, 3)
    train = dataset[:139]
    val = dataset[139:174]
    #278 - 5, 261 - 4
    #130 - 4, 

    X_train, y_train = zip(*train)
    X_val, y_val = zip(*val)

    X_train = np.array([x.reshape(input_shape) for x in X_train])
    X_val = np.array([x.reshape(input_shape) for x in X_val])

    y_train = np.array(np_utils.to_categorical(y_train, 2))
    y_val = np.array(np_utils.to_categorical(y_val, 2))
    
    return X_val, y_val 

In [5]:
def get_auc_values(models, X_val, y_val):
    base_fpr = np.linspace(0, 1, 100)
    
    tprs = []
    aucs = []

    for i in models:
        model = load_model(i, compile = True)
        y_score = model.predict(X_val)
        fpr, tpr, thresholds = roc_curve(y_val.argmax(axis=1), y_score[:,1])
        tprs.append(np.interp(base_fpr, fpr, tpr))
        tprs[-1][0] = 0.0
        roc_auc = auc(fpr, tpr)
        aucs.append(roc_auc)
        
    return tprs, aucs

In [6]:
def get_average_auc(aucs):
    Sum = 0 
    for i in aucs:
        Sum += i 
    
    return Sum/len(aucs)

In [12]:
def main(models, dataset):
    X_val, y_val = train_val(dataset)
    tprs, aucs = get_auc_values(models, X_val, y_val)
    print(aucs)
    score = get_average_auc(aucs)
    return score

In [38]:
score = main(models1, dataset)

2/2 [==============================] - 7s 703ms/step
[0.8636363636363636, 0.8846153846153846, 0.7062937062937064, 0.8776223776223776, 0.8601398601398601]


In [39]:
print(score)

0.8384615384615385


In [11]:
print(models1)

['/Users/home/Documents/Respiratory Illness AI/randomness_test/run0/asthma0_VGG19_1/', '/Users/home/Documents/Respiratory Illness AI/randomness_test/run0/asthma0_VGG19_2/', '/Users/home/Documents/Respiratory Illness AI/randomness_test/run0/asthma0_VGG19_5/', '/Users/home/Documents/Respiratory Illness AI/randomness_test/run0/asthma0_VGG19_4/', '/Users/home/Documents/Respiratory Illness AI/randomness_test/run0/asthma0_VGG19_3/']


In [ ]:
0.8411764705882353, 0.8156862745098039, 0.8741258741258742